<a href="https://colab.research.google.com/github/Shoaib013/CV/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-15 09:27:57--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-06-15 09:27:58 (181 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [11]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})
df_books.columns = ['isbn', 'title', 'author']
df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})
df_ratings.columns = ['user', 'isbn', 'rating']

In [12]:
df = pd.merge(df_books, df_ratings, on='isbn')

In [13]:
book_counts = df['title'].value_counts()
df = df[df['title'].isin(book_counts[book_counts >= 10].index)]

user_counts = df['user'].value_counts()
df = df[df['user'].isin(user_counts[user_counts >= 10].index)]

# Step 5: Build user-item matrix
user_item_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Step 6: Fit NearestNeighbors model
book_sparse = csr_matrix(user_item_matrix.values)

In [14]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
def get_recommends(book_title):
    if book_title not in user_item_matrix.index:
        return [book_title, []]

    book_idx = user_item_matrix.index.get_loc(book_title)
    distances, indices = model.kneighbors(book_sparse[book_idx], n_neighbors=6)

    # Build recommendations list
    recommendations = []
    for i in range(1, len(distances.flatten())):  # Skip first (itself)
        title = user_item_matrix.index[indices.flatten()[i]]
        score = distances.flatten()[i]  # Already cosine distance (lower is better)
        recommendations.append([title, score])

    return [book_title, recommendations]

In [19]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   np.float32(0.51347935)],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', np.float32(0.523026)],
  ['Interview with the Vampire', np.float32(0.64729357)],
  ['Memnoch the Devil (Vampire Chronicles, No 5)', np.float32(0.6782565)],
  ['Pandora: New Tales of the Vampires (New Tales of the Vampires)',
   np.float32(0.798036)]]]

In [20]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.8347904)], ['Forever Yours, Faithfully: My Love Story', np.float32(0.86066395)], ["Welcome to the Great Mysterious (Ballantine Reader's Circle)", np.float32(0.8608809)], ['I Know This Much Is True', np.float32(0.86189526)], ['Blue Diary', np.float32(0.86369133)]]]
You haven't passed yet. Keep trying!
